In [2]:
import os
import glob
import math
import numpy as np
import pandas as pd
import xarray as xr

import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
pastal = '/mnt/HD_500GB_WD/historia2/Tabelas_passo9/comtr-medianas-final/limpos/'
pastap = '/mnt/HD_500GB_WD/historia2/Tabelas_passo9/comtr-medianas-final/poluidos/'
pastat2 = '/mnt/HD_500GB_WD/historia2/Tabelas_passo9/comtr-medianas-final/trans2/'
pastat3 = '/mnt/HD_500GB_WD/historia2/Tabelas_passo9/comtr-medianas-final/trans3/'
pastat4 = '/mnt/HD_500GB_WD/historia2/Tabelas_passo9/comtr-medianas-final/trans4/'

In [4]:
dfsl = []
arquivos_l = sorted(glob.glob(pastal+'df_nuvenspasso9_??_limpa_??_??_2014.pkl'))
for j in range(len(arquivos_l)):
    n = arquivos_l[j].split('/')[-1].split('_')[-3]+'/'+arquivos_l[j].split('/')[-1].split('_')[-2]
    df = pd.read_pickle(arquivos_l[j])
    dfsl.append(df)

In [5]:
dfsp = []
arquivos_p = sorted(glob.glob(pastap+'df_nuvenspasso9_??_poluida_??_??_2014.pkl'))
for i in range(len(arquivos_p)):
    m = arquivos_p[i].split('/')[-1].split('_')[-3]+'/'+arquivos_p[i].split('/')[-1].split('_')[-2]
    df = pd.read_pickle(arquivos_p[i])
    dfsp.append(df)

In [6]:
dfst2 = []
arquivos_t2 = sorted(glob.glob(pastat2+'df_nuvenspasso9_??_trans2_??_??_2014.pkl'))
for j in range(len(arquivos_t2)):
    n = arquivos_t2[j].split('/')[-1].split('_')[-3]+'/'+arquivos_t2[j].split('/')[-1].split('_')[-2]
    df = pd.read_pickle(arquivos_t2[j])
    dfst2.append(df)
arquivos_t2b = sorted(glob.glob(pastat2+'df_nuvenspasso9_???_trans2_??_??_2014.pkl'))
for jb in range(len(arquivos_t2b)):
    n = arquivos_t2b[jb].split('/')[-1].split('_')[-3]+'/'+arquivos_t2b[jb].split('/')[-1].split('_')[-2]
    dfb = pd.read_pickle(arquivos_t2b[jb])
    dfst2.append(dfb)

In [7]:
dfst3 = []
arquivos_t3 = sorted(glob.glob(pastat3+'df_nuvenspasso9_??_trans3_??_??_2014.pkl'))
for j in range(len(arquivos_t3)):
    n = arquivos_t3[j].split('/')[-1].split('_')[-3]+'/'+arquivos_t3[j].split('/')[-1].split('_')[-2]
    df = pd.read_pickle(arquivos_t3[j])
    dfst3.append(df)

In [8]:
dfst4 = []
arquivos_t4 = sorted(glob.glob(pastat4+'df_nuvenspasso9_??_trans4_??_??_2014.pkl'))
for j in range(len(arquivos_t4)):
    n = arquivos_t4[j].split('/')[-1].split('_')[-3]+'/'+arquivos_t4[j].split('/')[-1].split('_')[-2]
    df = pd.read_pickle(arquivos_t4[j])
    dfst4.append(df)

In [9]:
#######################################################################

In [10]:
###################### MODELO 4 - Nuvens com 2 camadas, com o topo=percentil87.5(cth), a base=percentil12.5(cbh), 
###################### o meio=[base+((topo-base)/2)], lwctopo=percentil87.5(lwc),
###################### lwcbase=percentil12.5(lwc), lwcmeio=mediana(lwc) e a mesma 
###################### configuração para o reff. Aqui é gerado um dataframe para cada dia:
def lista_nuvens_modelo4(lista_com_dfs):
    dfs_rad = []
    for df in lista_com_dfs:
        stratus = df[(df['lwc']>=0.2) & (df['lwc']<=0.4) & (df['cth']<=3000.0)]
        if len(stratus) == 0:
#             print('0 stratus')
            pass
        elif len(stratus) == 1:
#             print('1 stratus')
            topo = stratus['cth']/1000
            base = stratus['cbh']/1000
            meio = base+((topo-base)/2)
            h = [float(math.ceil(topo)), float(topo), float(meio), float(base), float(math.floor(base))]
            dia = list(stratus['dia'])*5
            lwc = list(stratus['lwc'])*3
            ref = list(stratus['reff'])*3
            lwc.insert(0, 0.0)
            lwc.insert(4, 0.0)
            ref.insert(0, 0.0)
            ref.insert(4, 0.0)
            dic = {'dia':dia, 'z':h, 'LWC':lwc, 'R_eff':ref}
            df = pd.DataFrame(dic)
            dfs_rad.append(df)
        elif len(stratus) > 1:
#             print('mais de 1 stratus')
            dia = [stratus['dia'].iloc[0]]*5
            ztopo = np.percentile(stratus['cth']/1000, q=87.5)
            zbase = np.percentile(stratus['cbh']/1000, q=12.5)
            zmeio = zbase+((ztopo-zbase)/2)
            h = [float(math.ceil(ztopo)), float(ztopo), float(zmeio), float(zbase), float(math.floor(zbase))]
            stats = stratus.describe()
            lwctopo = np.percentile(stratus['lwc'], q=87.5)
            lwcmeio = stats['lwc'].loc['50%']
            lwcbase = np.percentile(stratus['lwc'], q=12.5)
            lwc = [0.0, lwctopo, lwcmeio, lwcbase, 0.0]
            reftopo = np.percentile(stratus['reff'], q=87.5)
            refmeio = stats['reff'].loc['50%']
            refbase = np.percentile(stratus['reff'], q=12.5)
            ref = [0.0, reftopo, refmeio, refbase, 0.0]
            dic = {'dia':dia, 'z':h, 'LWC':lwc, 'R_eff':ref}
            df = pd.DataFrame(dic)
            dfs_rad.append(df)
    return dfs_rad

In [11]:
dfs_limpa_rad = lista_nuvens_modelo4(dfsl)
dfs_poluida_rad = lista_nuvens_modelo4(dfsp)
dfs_trans2_rad = lista_nuvens_modelo4(dfst2)
dfs_trans3_rad = lista_nuvens_modelo4(dfst3)
dfs_trans4_rad = lista_nuvens_modelo4(dfst4)

In [12]:
###############################################################################

In [13]:
horaUTC = ' 16 00 00'

In [14]:
### Para o período limpo guardando as strings com os "times" dos dias e as strings com os nomes dos 
### arquivos de nuvens já da maneira que o LibRadtran pede: 
pasta_nuvens_stratus_modelo4 = '/home/andre/libRadtran-2.0.4/codigos_doutorado/nuvens4_lwcsemcal_FINAIS/limpos/'
timesl, indexesl = [], []
for i, df in enumerate(dfs_limpa_rad):
    dian = df['dia'][0]
    day = dian.split('/')[0]
    m = dian.split('/')[1]
    if i < 10:
        idx = '000{0}'.format(i)
    elif (i>=10) and (i<100):
        idx = '00{0}'.format(i)
    elif (i>=100) and (i<1000):
        idx = '0{0}'.format(i)
    elif i>=1000:
        idx = '{0}'.format(i)
    time = 'time 2014 '+m+' '+day+horaUTC+'\n'
    timesl.append(time)
    index = 'wc_file 1D '+pasta_nuvens_stratus_modelo4+'stratus_{0}_limpa_{1}_{2}_2014.csv\n'.format(idx,day,m)
    indexesl.append(index)
#     print(index)

In [15]:
### Para o período poluído guardando as strings com os "times" dos dias e as strings com os nomes dos 
### arquivos de nuvens já da maneira que o LibRadtran pede:
pasta_nuvens_stratus_modelo4 = '/home/andre/libRadtran-2.0.4/codigos_doutorado/nuvens4_lwcsemcal_FINAIS/poluidos/'
timesp, indexesp = [], []
for i, df in enumerate(dfs_poluida_rad):
    dian = df['dia'][0]
    day = dian.split('/')[0]
    m = dian.split('/')[1]
    if i < 10:
        idx = '000{0}'.format(i)
    elif (i>=10) and (i<100):
        idx = '00{0}'.format(i)
    elif (i>=100) and (i<1000):
        idx = '0{0}'.format(i)
    elif i>=1000:
        idx = '{0}'.format(i)
    time = 'time 2014 '+m+' '+day+horaUTC+'\n'
    timesp.append(time)
    index = 'wc_file 1D '+pasta_nuvens_stratus_modelo4+'stratus_{0}_poluida_{1}_{2}_2014.csv\n'.format(idx,day,m)
    indexesp.append(index)

In [16]:
### Para o período trans2 guardando as strings com os "times" dos dias e as strings com os nomes dos 
### arquivos de nuvens já da maneira que o LibRadtran pede: 
pasta_nuvens_stratus_modelo4 = '/home/andre/libRadtran-2.0.4/codigos_doutorado/nuvens4_lwcsemcal_FINAIS/trans2/'
timest2, indexest2 = [], []
for i, df in enumerate(dfs_trans2_rad):
    dian = df['dia'][0]
    day = dian.split('/')[0]
    m = dian.split('/')[1]
    if i < 10:
        idx = '000{0}'.format(i)
    elif (i>=10) and (i<100):
        idx = '00{0}'.format(i)
    elif (i>=100) and (i<1000):
        idx = '0{0}'.format(i)
    elif i>=1000:
        idx = '{0}'.format(i)
    time = 'time 2014 '+m+' '+day+horaUTC+'\n'
    timest2.append(time)
    index = 'wc_file 1D '+pasta_nuvens_stratus_modelo4+'stratus_{0}_trans2_{1}_{2}_2014.csv\n'.format(idx,day,m)
    indexest2.append(index)

In [17]:
### Para o período trans3 guardando as strings com os "times" dos dias e as strings com os nomes dos 
### arquivos de nuvens já da maneira que o LibRadtran pede: 
pasta_nuvens_stratus_modelo4 = '/home/andre/libRadtran-2.0.4/codigos_doutorado/nuvens4_lwcsemcal_FINAIS/trans3/'
timest3, indexest3 = [], []
for i, df in enumerate(dfs_trans3_rad):
    dian = df['dia'][0]
    day = dian.split('/')[0]
    m = dian.split('/')[1]
    if i < 10:
        idx = '000{0}'.format(i)
    elif (i>=10) and (i<100):
        idx = '00{0}'.format(i)
    elif (i>=100) and (i<1000):
        idx = '0{0}'.format(i)
    elif i>=1000:
        idx = '{0}'.format(i)
    time = 'time 2014 '+m+' '+day+horaUTC+'\n'
    timest3.append(time)
    index = 'wc_file 1D '+pasta_nuvens_stratus_modelo4+'stratus_{0}_trans3_{1}_{2}_2014.csv\n'.format(idx,day,m)
    indexest3.append(index)

In [18]:
### Para o período trans4 guardando as strings com os "times" dos dias e as strings com os nomes dos 
### arquivos de nuvens já da maneira que o LibRadtran pede: 
pasta_nuvens_stratus_modelo4 = '/home/andre/libRadtran-2.0.4/codigos_doutorado/nuvens4_lwcsemcal_FINAIS/trans4/'
timest4, indexest4 = [], []
for i, df in enumerate(dfs_trans4_rad):
    dian = df['dia'][0]
    day = dian.split('/')[0]
    m = dian.split('/')[1]
    if i < 10:
        idx = '000{0}'.format(i)
    elif (i>=10) and (i<100):
        idx = '00{0}'.format(i)
    elif (i>=100) and (i<1000):
        idx = '0{0}'.format(i)
    elif i>=1000:
        idx = '{0}'.format(i)
    time = 'time 2014 '+m+' '+day+horaUTC+'\n'
    timest4.append(time)
    index = 'wc_file 1D '+pasta_nuvens_stratus_modelo4+'stratus_{0}_trans4_{1}_{2}_2014.csv\n'.format(idx,day,m)
    indexest4.append(index)

In [19]:
###############################################################################

In [20]:
### Strings que devem entrar nos arquivos ".inp" que serão usados para fazer as simulações:
strings = ['####################_ATMOSFERA_####################\n', 
           'atmosphere_file ../data/atmmod/afglt.dat\n',
           '####################_ESPECTRO_####################\n',
           'source solar ../data/solar_flux/kurudz_0.1nm.dat\n',
           'mol_abs_param reptran medium\n',
           'rte_solver disort\n',
           'number_of_streams  16\n',
           'wavelength 300.0 1000.0\n',
           '####################_LOCAL_DE_ESTUDO_####################\n',
           'altitude 0.05\n',
           'longitude W 60 35 55\n',         
           'latitude S 03 12 47.82\n',
           'time 2014 02 22 16 00 00\n',
           'zout TOA\n',
           'albedo 0.14\n',   
           '####################_NUVENS_####################\n',
           'wc_file 1D /home/andre/libRadtran-2.0.4/codigos/nuvens_stratus_modelo3/limpos/stratus_0000_limpa_18_02_2014.csv\n',
           'cloud_overlap off\n',
           'cloudcover wc 1\n',
           '####################_GASES_####################\n',
           'mol_file N2O ../data/atmmod/afglt_n2o_vmr.csv vmr\n',
           'mol_file CH4 ../data/atmmod/afglt_ch4_vmr.csv vmr\n',
           'mol_file CO ../data/atmmod/afglt_co_vmr.csv vmr\n',
           'mixing_ratio CO2 397.0\n',
           '####################_OUTPUTS_####################\n',
           'output_user lambda zout albedo T T_sur TCC sza edir eup\n',
           'quiet\n']
opcionais = ['output_user lambda zout albedo T T_sur TCC sza edir eup\n', 'output_process integrate\n',
             'verbose\n']

In [21]:
### Pasta onde vou gravar os ".inp" para fazer as simulações no LibRadtran:
pasta_imports_limpa = '/home/andre/libRadtran-2.0.4/cc_inps_espectros_limpos-final/'
pasta_imports_poluida = '/home/andre/libRadtran-2.0.4/cc_inps_espectros_poluidos-final/'
pasta_imports_trans2 = '/home/andre/libRadtran-2.0.4/cc_inps_espectros_trans2-final/'
pasta_imports_trans3 = '/home/andre/libRadtran-2.0.4/cc_inps_espectros_trans3-final/'
pasta_imports_trans4 = '/home/andre/libRadtran-2.0.4/cc_inps_espectros_trans4-final/'

In [22]:
### Gravando os ".inp" dos dias de nuvens limpas:
for k in range(len(indexesl)):
    n = indexesl[k].split('/')[-1].split('.')[0].split('stratus_')[1]
    arquivo = open(pasta_imports_limpa+'import_{0}.inp'.format(n), 'w')
    for i in range(len(strings)):
        if (i<12):
            arquivo.write(strings[i])
        elif i==12:
            arquivo.write(timesl[k])
        elif (i>12) and (i<16):
            arquivo.write(strings[i])
        elif i==16:
            arquivo.write(indexesl[k])
        else:
            arquivo.write(strings[i])
    arquivo.close()

In [23]:
### Gravando os ".inp" dos dias de nuvens poluidas:
for j in range(len(indexesp)):
    n = indexesp[j].split('/')[-1].split('.')[0].split('stratus_')[1]
    arquivo = open(pasta_imports_poluida+'import_{0}.inp'.format(n), 'w')
    for i in range(len(strings)):
        if (i<12):
            arquivo.write(strings[i])
        elif i==12:
            arquivo.write(timesp[j])
        elif (i>12) and (i<16):
            arquivo.write(strings[i])
        elif i==16:
            arquivo.write(indexesp[j])
        else:
            arquivo.write(strings[i])
    arquivo.close()

In [24]:
### Gravando os ".inp" dos dias de nuvens trans2:
for k in range(len(indexest2)):
    n = indexest2[k].split('/')[-1].split('.')[0].split('stratus_')[1]
    arquivo = open(pasta_imports_trans2+'import_{0}.inp'.format(n), 'w')
    for i in range(len(strings)):
        if (i<12):
            arquivo.write(strings[i])
        elif i==12:
            arquivo.write(timest2[k])
        elif (i>12) and (i<16):
            arquivo.write(strings[i])
        elif i==16:
            arquivo.write(indexest2[k])
        else:
            arquivo.write(strings[i])
    arquivo.close()

In [25]:
### Gravando os ".inp" dos dias de nuvens trans3:
for k in range(len(indexest3)):
    n = indexest3[k].split('/')[-1].split('.')[0].split('stratus_')[1]
    arquivo = open(pasta_imports_trans3+'import_{0}.inp'.format(n), 'w')
    for i in range(len(strings)):
        if (i<12):
            arquivo.write(strings[i])
        elif i==12:
            arquivo.write(timest3[k])
        elif (i>12) and (i<16):
            arquivo.write(strings[i])
        elif i==16:
            arquivo.write(indexest3[k])
        else:
            arquivo.write(strings[i])
    arquivo.close()

In [26]:
### Gravando os ".inp" dos dias de nuvens trans4:
for k in range(len(indexest4)):
    n = indexest4[k].split('/')[-1].split('.')[0].split('stratus_')[1]
    arquivo = open(pasta_imports_trans4+'import_{0}.inp'.format(n), 'w')
    for i in range(len(strings)):
        if (i<12):
            arquivo.write(strings[i])
        elif i==12:
            arquivo.write(timest4[k])
        elif (i>12) and (i<16):
            arquivo.write(strings[i])
        elif i==16:
            arquivo.write(indexest4[k])
        else:
            arquivo.write(strings[i])
    arquivo.close()